# Day 24: RAG Optimization Implementation

In this notebook, we'll implement several optimization techniques to improve the performance, factuality, and traceability of our RAG system.

## Overview

We will cover:
1.  **Setup**: Re-establish our baseline RAG system.
2.  **Advanced Prompting**: Crafting better instructions for the LLM.
3.  **Context Re-ordering**: Mitigating the "lost in the middle" problem.
4.  **Relevance Filtering**: Removing irrelevant "noise" from the context.
5.  **Adding Citations**: Building a system that can cite its sources.

## 1. Setup

First, let's install libraries and set up our data. We'll use a slightly more complex dataset this time, split into multiple documents to make citations meaningful.

In [ ]:
!pip install sentence-transformers faiss-cpu openai python-dotenv rank_bm25

In [ ]:
import os
import openai
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# --- 1. Data Preparation with Metadata ---
documents = [
    {"source": "Zoltar_Geology.txt", "content": "The planet Zoltar is known for its crystalline forests. The 'Great Crystal' at the north pole is a massive geological formation believed to be the source of all life."},
    {"source": "Zoltarian_Biology.txt", "content": "Zoltarians are sentient, silicon-based lifeforms. They communicate using light patterns called 'Luminar'. They reproduce asexually by budding once every 50 Earth years."},
    {"source": "Zoltarian_Culture.txt", "content": "Zoltarian society is structured around the Great Crystal. Their diet consists of absorbing geothermal energy from volcanic vents."},
    {"source": "Zoltar_Astronomy.pdf", "content": "Zoltar has two suns, Helios Prime and Helios Beta, creating a perpetual twilight. Its atmosphere is mainly nitrogen and argon."}
]

# Store chunks and their metadata separately
text_chunks = [doc["content"] for doc in documents]
metadata = [doc["source"] for doc in documents]

# --- 2. Indexing ---
bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

chunk_embeddings = bi_encoder.encode(text_chunks, convert_to_tensor=False)
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(chunk_embeddings.astype('float32'))

print(f"Indexed {len(text_chunks)} documents.")

## 2. Advanced Prompting

Let's define a more robust prompt that instructs the model to answer only from the context.

In [ ]:
def create_advanced_prompt(query, context):
    return (
        f"You are a helpful assistant. Answer the user's question based ONLY on the provided context.\n"
        f"If the answer is not found in the context, you MUST say 'I'm sorry, the context does not contain the answer.'\n"
        f"Do not use any external knowledge.\n\n"
        f"Context:\n---\n{context}\n---\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )

## 3. Context Re-ordering for Long Contexts

This function re-orders documents to place the most relevant ones at the beginning and end of the context, combating the "lost in the middle" problem.

In [ ]:
def reorder_documents(doc_indices):
    """Re-orders docs to place important ones at the beginning and end."""
    reordered = []
    while doc_indices:
        # Add the most relevant doc
        reordered.append(doc_indices.pop(0))
        # If there are more docs, add the next most relevant to the end
        if doc_indices:
            reordered.insert(0, doc_indices.pop(0))
    return reordered

## 4. Relevance Filtering & Citations

Now, let's build a final, optimized RAG pipeline that incorporates reranking, relevance filtering, context re-ordering, and citations.

In [ ]:
def optimized_rag_pipeline(query, k=4):
    # 1. Initial Retrieval (Bi-Encoder)
    query_embedding = bi_encoder.encode([query])
    _, initial_indices = index.search(query_embedding.astype('float32'), k * 2) # Retrieve more for reranking
    initial_indices = initial_indices[0].tolist()

    # 2. Reranking (Cross-Encoder)
    pairs = [(query, text_chunks[i]) for i in initial_indices]
    scores = cross_encoder.predict(pairs)
    scored_indices = sorted(zip(initial_indices, scores), key=lambda x: x[1], reverse=True)

    # 3. Relevance Filtering
    relevance_threshold = 0.1 # Example threshold
    filtered_indices = [idx for idx, score in scored_indices if score > relevance_threshold]

    # 4. Context Re-ordering
    final_indices = reorder_documents(filtered_indices[:k])

    # 5. Build Context with Citations
    context_parts = []
    for i, idx in enumerate(final_indices):
        source = metadata[idx]
        content = text_chunks[idx]
        context_parts.append(f"Source [{i+1}]: {source}\nContent: {content}")
    context = "\n\n---\n\n".join(context_parts)

    # 6. Advanced Prompting
    prompt = (
        f"You are an expert assistant. Answer the user's question based ONLY on the provided sources. \n"
        f"Cite the source number (e.g., [Source 1]) for each piece of information you use.\n\n"
        f"Sources:\n{context}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )

    # 7. Generation
    print("--- Final Prompt to LLM ---")
    print(prompt)
    final_answer = 'Simulated LLM response' # get_llm_response(prompt) - Mocked for this example
    if not openai.api_key:
        if 'communicate' in query:
            final_answer = "Zoltarians communicate using light patterns known as 'Luminar' [Source 2]."
        elif 'eat' in query:
            final_answer = "The Zoltarian diet involves absorbing geothermal energy from volcanic vents [Source 1]."
        else:
            final_answer = "I'm sorry, the context does not contain the answer."

    return final_answer

# --- Test the optimized pipeline ---
query1 = "How do Zoltarians communicate and what do they eat?"
print(f'\n>>> Query: {query1}')
answer1 = optimized_rag_pipeline(query1)
print(f'\n>>> Final Answer: {answer1}')

print('\n' + '='*50 + '\n')

# Test a query where the answer is not in the context
query2 = "What is the capital city of Zoltar?"
print(f'>>> Query: {query2}')
answer2 = optimized_rag_pipeline(query2)
print(f'\n>>> Final Answer: {answer2}')

## 5. Conclusion

In this notebook, we have implemented several key strategies to optimize a RAG pipeline:

-   **Advanced Prompting**: We created a detailed prompt that constrains the model to use only the provided context, improving factuality.
-   **Context Re-ordering**: By placing the most relevant documents at the beginning and end of the context, we mitigate the "lost in the middle" problem and improve the model's attention.
-   **Relevance Filtering**: Using a reranker's score as a threshold helps us filter out noisy, irrelevant documents before they ever reach the LLM.
-   **Citations**: By including document metadata in the prompt and instructing the model to use it, we've built a system that can trace its answers back to the source, dramatically increasing trustworthiness.

These techniques, when combined, lead to a RAG system that is more accurate, reliable, and transparent.